In [25]:
import os
import pandas as pd
import seaborn as sns
from pathlib import Path
import numpy as np

def set_correct_working_dir(working_dir: str):
    current_working_dir = Path(os.getcwd())
    if working_dir == current_working_dir.name:
        print('WD correct')
        return
    for directory in current_working_dir.parents:
        if working_dir == directory.name:
            os.chdir(str(directory.absolute()))
            print('New WD:', str(directory.absolute()))
            return
    raise FileNotFoundError

set_correct_working_dir('geo-locations')

WD correct


### Reading in data

In [77]:
country_code = 'che'
country_path = Path(os.getcwd()) / 'data' / country_code 
source = country_path / 'source' / 'PLZO_CSV_WGS84' / 'PLZO_CSV_WGS84.csv'
# column description to be found at: https://download.geonames.org/export/zip/
che_df = pd.read_csv(source, sep=';', encoding='latin_1', converters={'PLZ': str})
display(che_df.head())

,Ortschaftsname,PLZ,Zusatzziffer,Gemeindename,BFS-Nr,Kantonskürzel,E,N,Sprache
0,Aeugst am Albis,8914,0,Aeugst am Albis,1,ZH,8.488313,47.267004,de
1,Aeugstertal,8914,2,Aeugst am Albis,1,ZH,8.493642,47.282761,de
2,Zwillikon,8909,0,Affoltern am Albis,2,ZH,8.431459,47.287633,de
3,Affoltern am Albis,8910,0,Affoltern am Albis,2,ZH,8.448945,47.279169,de
4,Bonstetten,8906,0,Bonstetten,3,ZH,8.467611,47.315510,de


In [80]:
try:
    che_df.drop(columns=['Zusatzziffer', 'BFS-Nr', 'Sprache'], inplace=True)
except KeyError:
    pass
try:
    che_df.rename(columns={'PLZ': 'postal_code', 'Ortschaftsname': 'place_name', 'Gemeindename': 'admin_name_2', 'Kantonskürzel': 'admin_name_1', 'E': 'latitude', 'N': 'longitude'}, inplace=True)
except KeyError:
    pass

che_df.fillna(np.NaN)


che_df = che_df.groupby('postal_code').agg({'admin_name_1': lambda col: ';'.join(col),
                                            'admin_name_2': lambda col: ';'.join(col),
                                            'place_name': lambda col: '||'.join(col),
                                            'latitude': 'mean',
                                            'longitude': 'mean'}).reset_index()

che_df.loc[((che_df['admin_name_2'] == che_df['place_name']) & (che_df['admin_name_2'].count() > 1), 'place_name')] = che_df['postal_code']
che_df.loc[(che_df['admin_name_2'].str.split(';') == che_df['place_name'].str.split('\|\|'), 'admin_name_2')] = ''

che_df['admin_name_1'] = che_df['admin_name_1'].apply(lambda x: ';'.join(list(set(x.split(';')))) if ';' in x else x)
che_df['admin_name_2'] = che_df['admin_name_2'].apply(lambda x: ';'.join(list(set(x.split(';')))) if ';' in x else x)
che_df['region_id'] = che_df['admin_name_1'] + '::' + che_df['admin_name_2'] + '::' + che_df['place_name']
che_df.loc[che_df['admin_name_2'] == '', 'region_id'] = che_df['admin_name_1'] + '::' + che_df['place_name']
che_df.reset_index(inplace=True, drop=True)
che_df.sort_values('postal_code', inplace=True)
che_df.insert(loc=0, column='country_code', value=country_code)
che_df = che_df.round({'latitude': 4, 'longitude': 4})
display(che_df.head(50))

,country_code,postal_code,admin_name_1,admin_name_2,place_name,latitude,longitude,region_id
0,che,1000,VD,Lausanne,Lausanne 25||Lausanne 26||Lausanne 27,6.6870,46.5535,VD::Lausanne::Lausanne 25||Lausanne 26||Lausan...
1,che,1003,VD,Lausanne,1003,6.6300,46.5200,VD::Lausanne::1003
2,che,1004,VD,Lausanne,1004,6.6187,46.5285,VD::Lausanne::1004
3,che,1005,VD,Lausanne,1005,6.6425,46.5199,VD::Lausanne::1005
4,che,1006,VD,Lausanne,1006,6.6371,46.5108,VD::Lausanne::1006
5,che,1007,VD,Lausanne,1007,6.6086,46.5178,VD::Lausanne::1007
6,che,1008,VD,,Jouxtens-Mézery||Prilly,6.6027,46.5436,VD::Jouxtens-Mézery||Prilly
7,che,1009,VD,Pully,1009,6.6620,46.5092,VD::Pully::1009
8,che,1010,VD,Lausanne,1010,6.6589,46.5361,VD::Lausanne::1010
9,che,1011,VD,Lausanne,1011,6.6429,46.5256,VD::Lausanne::1011


In [83]:
try:
    che_df.drop(columns=['admin_name_1', 'admin_name_2','place_name'], inplace=True)
except KeyError:
    pass
file_path = country_path / '_'.join([country_code, 'geocoding_sample.csv'])
che_df.to_csv(file_path, index=False)